In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import json
import requests
import pandas as pd
import pyodbc
import openai
from rich.console import Console
from rich.panel import Panel
from dotenv import load_dotenv

load_dotenv()
console = Console()

# Azure OpenAI configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-azure-openai-endpoint.openai.azure.com/")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME", "gpt-4o")  # update as needed

# Azure AI Search configuration
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-search-service.search.windows.net")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY", "your-azure-search-key")
SEARCH_INDEX_NAME = "acc-guidelines-index"


# Bing Search credentials (optional)
BING_SEARCH_API_KEY = os.getenv("BING_SEARCH_API_KEY", "<YOUR-BING-SEARCH-KEY>")
BING_SEARCH_ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"

# Import Azure libraries for search
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizableTextQuery

# Import Azure OpenAI client
from openai import AzureOpenAI

console = Console()

# Azure SQL connection details (for patient data)
server = os.getenv("AZURE_SQL_SERVER_NAME")
database = os.getenv("AZURE_SQL_DATABASE_NAME")
username = os.getenv("AZURE_SQL_USER_NAME")
password = os.getenv("AZURE_SQL_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

# Create an Azure SQL connection string
AZURE_SQL_CONNECTION_STRING = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"


In [3]:
# Initialize the Azure OpenAI client
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizableTextQuery

def search_acc_guidelines(query: str) -> str:
    """
    Searches the Azure AI Search index 'acc-guidelines-index' 
    for relevant American College of Cardiology (ACC) guidelines.
    """
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=credential,
    )
    
    results = client.search(
        search_text=query,
        vector_queries=[
            VectorizableTextQuery(
                text=query,
                k_nearest_neighbors=10,   # Adjust as needed
                fields="embedding"   # Adjust based on your index schema
            )
        ],
        query_type="semantic",
        semantic_configuration_name="default",
        search_fields=["chunk"],
        top=10,
        include_total_count=True
    )
    
    retrieved_texts = []
    for result in results:
        content_chunk = result.get("chunk", "")
        retrieved_texts.append(content_chunk)
    
    context_str = "\n".join(retrieved_texts) if retrieved_texts else "No relevant guidelines found."
    
    console.print(
        Panel(
            f"Tool Invoked: ACC Guidelines Search\nQuery: {query}",
            style="bold yellow"
        )
    )
    return context_str


In [5]:
def search_bing(query: str) -> str:
    """
    Searches the public web using the Bing Search API.
    """
    headers = {"Ocp-Apim-Subscription-Key": BING_SEARCH_API_KEY}
    params = {"q": query, "textDecorations": True, "textFormat": "Raw"}
    response = requests.get(BING_SEARCH_ENDPOINT, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if "webPages" in data and "value" in data["webPages"]:
            snippets = [item.get("snippet", "") for item in data["webPages"]["value"]]
            result_text = "\n".join(snippets)
        else:
            result_text = "No Bing results found."
    else:
        result_text = f"Bing search failed with status code {response.status_code}."
    
    console.print(
        Panel(
            f"Tool Invoked: Bing Search\nQuery: {query}",
            style="bold magenta"
        )
    )
    return result_text


In [54]:
import sqlalchemy

def lookup_patient_data(query: str) -> str:
    """
    Queries the 'PatientMedicalData' table in Azure SQL and returns the results as a string.
    'query' should be a valid SQL statement.
    This version uses SQLAlchemy to create an engine, which is fully supported by pandas.read_sql.
    """
    try:
        # Construct the connection URI for SQLAlchemy
        connection_uri = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = sqlalchemy.create_engine(connection_uri)
        
        # Use the engine in pandas.read_sql, which avoids the warning
        df = pd.read_sql(query, engine)
        if df.empty:
            return "No rows found."
        return df.to_string(index=False)
    except Exception as e:
        return f"Database error: {str(e)}"


In [55]:
tools = [
    {
        "name": "search_acc_guidelines",
        "description": "Query the ACC guidelines for official cardiology recommendations.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The cardiology-related question or keywords."
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "search_bing",
        "description": "Perform a public web search for real-time or external information.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "General query to retrieve public data."
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "lookup_patient_data",
        "description": (
            "Query the PatientMedicalData table in Azure SQL. "
            "The table schema is as follows:\n\n"
            "PatientID: INT PRIMARY KEY IDENTITY,\n"
            "FirstName: VARCHAR(100),\n"
            "LastName: VARCHAR(100),\n"
            "DateOfBirth: DATE,\n"
            "Gender: VARCHAR(20),\n"
            "ContactNumber: VARCHAR(100),\n"
            "EmailAddress: VARCHAR(100),\n"
            "Address: VARCHAR(255),\n"
            "City: VARCHAR(100),\n"
            "PostalCode: VARCHAR(20),\n"
            "Country: VARCHAR(100),\n"
            "MedicalCondition: VARCHAR(255),\n"
            "Medications: VARCHAR(255),\n"
            "Allergies: VARCHAR(255),\n"
            "BloodType: VARCHAR(10),\n"
            "LastVisitDate: DATE,\n"
            "SmokingStatus: VARCHAR(50),\n"
            "AlcoholConsumption: VARCHAR(50),\n"
            "ExerciseFrequency: VARCHAR(50),\n"
            "Occupation: VARCHAR(100),\n"
            "Height_cm: DECIMAL(5,2),\n"
            "Weight_kg: DECIMAL(5,2),\n"
            "BloodPressure: VARCHAR(20),\n"
            "HeartRate_bpm: INT,\n"
            "Temperature_C: DECIMAL(3,1),\n"
            "Notes: VARCHAR(MAX)\n\n"
            "Generate and execute a safe SQL query based on the user's natural language request."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "A valid SQL query to run against the PatientMedicalData table."
                }
            },
            "required": ["query"]
        }
    }
]

# For convenience, map the tool name to the actual function
tool_implementations = {
    "search_acc_guidelines": search_acc_guidelines,
    "search_bing": search_bing,
    "lookup_patient_data": lookup_patient_data,
}

console = Console()

In [56]:
SYSTEM_PROMPT = (
    "You are a cardiology-focused AI assistant with access to three tools:\n"
    "1) 'search_acc_guidelines' for official ACC guidelines.\n"
    "2) 'search_bing' for real-time public information.\n"
    "3) 'lookup_patient_data' for patient data from Azure SQL.\n\n"
    "You can call these tools in any order, multiple times if needed, to gather all the context.\n"
    "Stop calling tools only when you have enough information to provide a final, cohesive answer.\n"
    "Then output your final answer to the user."
)

In [60]:
def run_multi_step_agent(user_query: str, max_steps: int = 5):
    """
    A multi-step agent that calls multiple tools in sequence until a final answer is provided
    or the max_steps limit is reached.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    for step in range(max_steps):
        response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            functions=tools,
            function_call="auto"
        )
        response_message = response.choices[0].message

        if response_message.function_call is not None:
            function_name = response_message.function_call.name
            arguments_str = response_message.function_call.arguments
            if arguments_str.strip() == "":
                function_args = {}
            else:
                try:
                    function_args = json.loads(arguments_str)
                except json.JSONDecodeError:
                    function_args = {}
                    console.print(
                        Panel(
                            "Warning: Could not decode function call arguments; defaulting to empty dict.",
                            title="JSONDecodeError",
                            style="bold red"
                        )
                    )
            # Ensure the required 'query' key is present; if not, default it to user_query.
            if "query" not in function_args or not function_args["query"]:
                function_args["query"] = user_query

            console.print(
                Panel(
                    f"**Step {step+1}**: LLM calls function [bold]{function_name}[/bold]\n\n"
                    f"**Arguments**:\n{json.dumps(function_args, indent=2)}",
                    title="Function Call",
                    style="bold blue"
                )
            )

            tool_fn = tool_implementations.get(function_name)
            if tool_fn is None:
                tool_output = f"[Error] No implementation for function '{function_name}'."
            else:
                tool_output = tool_fn(**function_args)

            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": tool_output
                }
            )
        else:
            final_answer = response_message.content
            console.print(
                Panel(
                    final_answer,
                    title="Final Answer",
                    style="bold green",
                    border_style="yellow"
                )
            )
            return

    console.print(
        Panel(
            "Max steps reached without a final answer. Stopping.",
            title="Warning",
            style="bold red"
        )
    )
    return


In [61]:
user_question_1 = "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"
run_multi_step_agent(user_question_1)


╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 1**: LLM calls function search_acc_guidelines                                                            │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "first-line therapy for hypertension in elderly patients"                                            │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: ACC Guidelines Search                                                                             │
│ Query: first-line therapy for hypertension in elderly patients                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ The American College of Cardiology (ACC) recommends the following first-line medications for the treatment of   │
│ hypertension in elderly patients:                                                                               │
│                                                                                                                 │
│ 1. **Thiazide diuretics** (e.g., chlorthalidone),                                                               │
│ 2. **Calcium channel blockers (CCBs)**, and                                                                     │
│ 3. **Angiotensin-converting enzyme (ACE) inhibitors** or **Angiotensin II receptor blockers (ARBs)**.           │
│                                                                                                                 │
│ It's essential to consider the patient's comorbid conditions when selecting the initial medication, as certain  │
│ classes may be more beneficial based on specific health issues (e.g., heart failure, chronic kidney disease).   │
│                                                                                                                 │
│ The guidelines suggest that a significant number of patients may benefit from initiating treatment with a       │
│ combination of two agents, especially if they have stage 2 hypertension or a blood pressure significantly above │
│ the target level. Close monitoring is advised, particularly in older adults, to avoid potential adverse effects │
│ like hypotension or orthostatic hypotension.                                                                    │
│                                                                                                                 │
│ Overall, the goal of treatment remains to reduce blood pressure while addressing the underlying cardiovascular  │
│ risk factors.                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [62]:
user_question_2 = "Are there any recent updates in 2025 on new anticoagulant therapies from the FDA?"
run_multi_step_agent(user_question_2)


╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 1**: LLM calls function search_bing                                                                      │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "2025 FDA updates new anticoagulant therapies"                                                       │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: 2025 FDA updates new anticoagulant therapies                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ As of early 2025, there are several updates regarding new anticoagulant therapies anticipated for FDA approval. │
│ Some of the notable points include:                                                                             │
│                                                                                                                 │
│ 1. **New Anti-Factor XI Anticoagulants**: There is ongoing research and development focused on new              │
│ anticoagulants that inhibit factor XI/XIa. These drugs are showing promise for preventing ischemic risks and    │
│ could represent a significant advancement in anticoagulant therapy.                                             │
│                                                                                                                 │
│ 2. **Anticipated Approvals**: The FDA is expected to review and possibly approve new medications in this area,  │
│ which might include novel anticoagulants that are not yet on the market or expanding the indications of         │
│ existing ones.                                                                                                  │
│                                                                                                                 │
│ 3. **Drug Approval Timeline**: The PDUFA (Prescription Drug User Fee Act) dates indicate that the FDA is likely │
│ to make decisions on various new drug applications throughout 2025. This means that several new anticoagulants  │
│ may soon be available or have expanded uses.                                                                    │
│                                                                                                                 │
│ These updates highlight the progress in anticoagulant therapy, particularly towards addressing the balance      │
│ between efficacy and safety, which remains a pivotal concern in anticoagulant use.                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [63]:
user_question_3 = "How many patients have Hypertension and are prescribed Lisinopril?"
# The agent should generate a valid SQL query, for example:
## Note, the answer will depend on the actual data in the database (it should return 1071!)
run_multi_step_agent(user_question_3)


╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 1**: LLM calls function lookup_patient_data                                                              │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "SELECT COUNT(*) AS PatientCount FROM PatientMedicalData WHERE MedicalCondition LIKE                 │
│ '%Hypertension%' AND Medications LIKE '%Lisinopril%'"                                                           │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ There are 1,071 patients who have Hypertension and are prescribed Lisinopril.                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [64]:
run_multi_step_agent("I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there are any new medication updates from the FDA? Then give me a summary.", max_steps=5)


╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 1**: LLM calls function lookup_patient_data                                                              │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "SELECT * FROM PatientMedicalData WHERE FirstName = 'Gloria' AND LastName = 'Paul'"                  │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 2**: LLM calls function search_acc_guidelines                                                            │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "hyperlipidemia guidelines"                                                                          │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: ACC Guidelines Search                                                                             │
│ Query: hyperlipidemia guidelines                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── JSONDecodeError ────────────────────────────────────────────────╮
│ Warning: Could not decode function call arguments; defaulting to empty dict.                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Function Call ─────────────────────────────────────────────────╮
│ **Step 3**: LLM calls function search_bing                                                                      │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you  │
│ confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there    │
│ are any new medication updates from the FDA? Then give me a summary."                                           │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you       │
│ confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there    │
│ are any new medication updates from the FDA? Then give me a summary.                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here's a summary of the information regarding your patient, Gloria Paul, and the guidelines for managing        │
│ hyperlipidemia:                                                                                                 │
│                                                                                                                 │
│ ### Patient Information                                                                                         │
│ - **Name**: Gloria Paul                                                                                         │
│ - **Age**: 79 years                                                                                             │
│ - **Medical Condition**: Hyperlipidemia                                                                         │
│ - **Current Medication**: Atorvastatin                                                                          │
│ - **Last Visit Date**: March 26, 2024                                                                           │
│ - **Blood Pressure**: 120/79 mmHg                                                                               │
│ - **Heart Rate**: 90 bpm                                                                                        │
│ - **Height**: 176.2 cm                                                                                          │
│ - **Weight**: 77.7 kg                                                                                           │
│ - **Allergies**: None                                                                                           │
│                                                                                                                 │
│ ### ACC Guidelines on Hyperlipidemia                                                                            │
│ According to the **2018 ACC/AHA Cholesterol Treatment Guidelines**, the following recommendations apply to      │
│ hyperlipidemia:                                                                                                 │
│ 1. **Statin Therapy**: Statins, like atorvastatin, are recommended for all patients with known atherosclerotic  │
│ cardiovascular disease (ASCVD) to lower LDL cholesterol levels.                                                 │
│    - **Class I Recommendation**: For patients with extracranial carotid or vertebral atherosclerosis, statin    │
│ treatment should aim to achieve LDL cholesterol levels below 100 mg/dL.                                         │
│    - **Class IIa Recommendation**: In patients who have had an ischemic stroke, it’s suggested to lower LDL     │
│ cholesterol to levels near or below 70 mg/dL if possible.                                                       │
│                                                                                                                 │
│ 2. **Combination Therapy**: If LDL cholesterol goals are not achieved with statins alone, combination therapy   │
│ may include medications such as ezetimibe or bile acid sequestrants.                                            │
│                                                                                                                 │
│ 3. **Lifestyle Modifications**: Addressing lifestyle factors (diet and exercise) is critical alongside          │
│ medication management to control cholesterol levels effectively.                                                │
│                                                                                                                 │
│ ### Recent Medication Updates from the FDA                                                                      │
│ A web search did not return new medications specifically announced for the treatment of hyperlipidemia beyond   │
│ existing therapies like statins and new agents like PC